In [ ]:
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from PIL import Image

In [ ]:
base_dir = './data/poses/center_pose1'

In [ ]:
os.path.exists(base_dir)

In [ ]:
intrinsic_mat = np.load(os.path.join(base_dir, 'camera_intrinsics.npz'))
intrinsic_mat

In [ ]:
fx, fy = intrinsic_mat['fx'], intrinsic_mat['fy']
ppx, ppy = intrinsic_mat['ppx'], intrinsic_mat['ppy']

In [ ]:
print(fx, fy, ppx, ppy)

In [ ]:
depth_image_path = os.path.join(base_dir, 'camera_depth')
depth_image_path_list = sorted([os.path.join(depth_image_path, f) for f in os.listdir(depth_image_path) if f.endswith('.npy')])
depth_image_list = [np.load(f) for f in depth_image_path_list]

In [ ]:
rgb_image_path = os.path.join(base_dir, 'camera_rgb')
rgb_image_path_list = sorted([os.path.join(rgb_image_path, f) for f in os.listdir(rgb_image_path) if f.endswith('.png')])
rgb_image_list = [np.array(Image.open(f).convert('RGB')) for f in rgb_image_path_list]

In [ ]:
assert len(depth_image_list) == len(rgb_image_list)
assert depth_image_list[0].shape == rgb_image_list[0].shape[:2]

In [ ]:
plt.imshow(rgb_image_list[0])

In [ ]:
cam_to_gripper_rot = np.array([[-7.77766820e-02, -9.44073658e-01, -3.20430518e-01],
 [ 9.96970509e-01, -7.34019190e-02, -2.57286360e-02],
 [ 7.69512542e-04, -3.21460864e-01,  9.46922553e-01]])

cam_to_gripper_trans = np.array([[ 0.10598264], [-0.02439176], [-0.08322135]])

In [ ]:
cam_to_gripper_pose = np.eye(4)
cam_to_gripper_pose[:3, :3] = cam_to_gripper_rot
cam_to_gripper_pose[:3, 3] = cam_to_gripper_trans.squeeze()

In [ ]:
cam_to_gripper_pose

In [ ]:
pose_path = os.path.join(base_dir, 'poses')
pose_path_list = sorted([os.path.join(pose_path, f) for f in os.listdir(pose_path) if f.endswith('.npy')])
pose_list = [np.load(f) @ cam_to_gripper_pose for f in pose_path_list]

In [ ]:
volume = o3d.pipelines.integration.ScalableTSDFVolume(
    voxel_length=0.01,
    sdf_trunc=3 * 0.01,
    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
)

H, W = depth_image_list[0].shape
fx, fy, cx, cy = intrinsic_mat['fx'], intrinsic_mat['fy'], intrinsic_mat['ppx'], intrinsic_mat['ppy']

for idx in trange(len(depth_image_list)):
    pose = pose_list[idx]

    rgb = rgb_image_list[idx]
    rgb = np.ascontiguousarray(rgb)
    depth = depth_image_list[idx] / 1000.0
    depth[depth > 5.0] = 0.0 # remove invalid depth
    depth = np.ascontiguousarray(depth.astype(np.float32))

    rgb = o3d.geometry.Image(rgb)
    depth = o3d.geometry.Image(depth)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, depth, depth_scale=1.0, depth_trunc=4.0, convert_rgb_to_intensity=False)
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width=W, height=H, fx=fx, fy=fy, cx=cx, cy=cy)
    extrinsic = np.linalg.inv(pose)
    # extrinsic = pose
    volume.integrate(rgbd, intrinsic, extrinsic)

In [ ]:
mesh = volume.extract_triangle_mesh()

In [ ]:
# visualize mesh
o3d.visualization.draw_geometries([mesh])

In [ ]:
mesh = volume.extract_triangle_mesh()
mesh_path = os.path.join('/tmp/tmp.ply')
o3d.io.write_triangle_mesh(mesh_path, mesh)